#### 데이터 로드 및 살펴보기

In [1]:
import pandas as pd

In [3]:
# 압축해제된 파일들을 열어보면 csv형태로 들어가 있다
# 고객 정보 로드
# 고객 정보 컬럼 : 아이디, 나이, 성별, 직업, 주소(우편번호)
cols = ['uid','age','sex','job','zip_code']
users = pd.read_csv( './ml-100k/u.user', sep='|', names=cols )
users.shape

(943, 5)

In [7]:
users.head()

,uid,age,sex,job,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [10]:
# 영화 정보 로드
# 영화ID, 제목, 개봉일, 비디오개봉일, imdb 주소
m_cols = ['mid','title','release_date','video_release_date','imdb_url']
# 실제 데이터에 컬럼이 더 많은데, 앞에서부터 잘라서 적용하고 싶다
movies = pd.read_csv( './ml-100k/u.item', sep='|', names=m_cols, encoding='latin1', usecols=range(5) ) #usecols= : 필요없는 것 버리고 5개의 컬럼을 형성한다
movies.head()

,mid,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)
